![Roboflow Notebooks banner](https://camo.githubusercontent.com/aec53c2b5fb6ed43d202a0ab622b58ba68a89d654fbe3abab0c0cc8bd1ff424e/68747470733a2f2f696b2e696d6167656b69742e696f2f726f626f666c6f772f6e6f7465626f6f6b732f74656d706c6174652f62616e6e657274657374322d322e706e673f696b2d73646b2d76657273696f6e3d6a6176617363726970742d312e342e33267570646174656441743d31363732393332373130313934)

# Image Classification with DINOv2

DINOv2, released by Meta Research in April 2023, implements a self-supervised method of training computer vision models.

DINOv2 was trained using 140 million images without labels. The embeddings generated by DINOv2 can be used for classification, image retrieval, segmentation, and depth estimation. With that said, Meta Research did not release heads for segmentation and depth estimation.
In this guide, we are going to build an image classifier using embeddings from DINOv2. To do so, we will:

1. Load a folder of images
2. Compute embeddings for each image
3. Save all the embeddings in a file and vector store
4. Train an SVM classifier to classify images

By the end of this notebook, we'll have a classifier trained on our dataset.

Without further ado, let's begin!

## Import Packages

First, let's import the packages we will need for this project.

In [48]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import os
#import cv2
import json
import glob
from tqdm.notebook import tqdm

In [49]:
cwd = os.getcwd()
cwd

'/workspaces/gc_quant_trading_research'

Load folder containing the trading images

In [50]:
cwd = os.getcwd()

ROOT_DIR = os.path.join(cwd)

labels = {}
# remove the folder Now from the for loop

for folder in os.listdir(ROOT_DIR):
  if folder == 'test':
    continue 
  else:
    try:
      print(folder)
      for file in os.listdir(os.path.join(ROOT_DIR, folder)):
          if file.endswith(".png"):
              full_name = os.path.join(ROOT_DIR, folder, file)
              labels[full_name] = folder
    except:
      pass

files = labels.keys()

gc_trading_1h.csv
2D_embedding_viz.ipynb
all_embeddings.json
candlestick_images
test.ipynb
bear
range
# %% [markdown]
Dinov2_classification_gc.ipynb
classification_app.py
2d_scatter_plot_4candles.html
2d_scatter_plot.html
bull
README.md
models
ml_experiment_config.toml
dockerfile
notebooks
.git
training_data_generator.py
data
requirements.txt


In [51]:
list(files)

['/workspaces/gc_quant_trading_research/bear/chart_3292.png',
 '/workspaces/gc_quant_trading_research/bear/chart_3443.png',
 '/workspaces/gc_quant_trading_research/bear/chart_1659.png',
 '/workspaces/gc_quant_trading_research/bear/chart_1087.png',
 '/workspaces/gc_quant_trading_research/bear/chart_2445.png',
 '/workspaces/gc_quant_trading_research/bear/chart_3075.png',
 '/workspaces/gc_quant_trading_research/bear/chart_2291.png',
 '/workspaces/gc_quant_trading_research/bear/chart_417.png',
 '/workspaces/gc_quant_trading_research/bear/chart_3212.png',
 '/workspaces/gc_quant_trading_research/bear/chart_959.png',
 '/workspaces/gc_quant_trading_research/bear/chart_2399.png',
 '/workspaces/gc_quant_trading_research/bear/chart_1741.png',
 '/workspaces/gc_quant_trading_research/bear/chart_2400.png',
 '/workspaces/gc_quant_trading_research/bear/chart_2662.png',
 '/workspaces/gc_quant_trading_research/bear/chart_846.png',
 '/workspaces/gc_quant_trading_research/bear/chart_1575.png',
 '/workspac

In [52]:
# prompt: get data from dictionary files

values = [labels[key] for key in files]

## Load the Model and Compute Embeddings

To train our classifier, we need:

1. The embeddings associated with each image in our dataset, and;
2. The labels associated with each image.

To calculate embeddings, we'll use DINOv2. Below, we load the smallest DINOv2 weights and define functions that will load and compute embeddings for every image in a specified list.

We store all of our vectors in a dictionary that is saved to disk so we can reference them again if needed. Note that in production environments one may opt for using another data structure such as a vector embedding database (i.e. faiss) for storing embeddings.

In [53]:
MODEL = "dinov2_vits14"
EMBEDDING_SIZE = 384 # 768 for vitb14, 384 for vits14

dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", MODEL)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

dinov2_vits14.to(device)

transform_image = T.Compose([T.ToTensor(),
                             T.Resize((70, 210)),
                             #T.CenterCrop(224),
                             T.Normalize([0.5], [0.5])])

Using cache found in /home/codespace/.cache/torch/hub/facebookresearch_dinov2_main


In [54]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """
    img = Image.open(img)

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}

    with torch.no_grad():
      for i, file in enumerate(files):
        embeddings = dinov2_vits14(load_image(file).to(device))

        all_embeddings[file] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

    with open("all_embeddings.json", "w") as f:
        f.write(json.dumps(all_embeddings))

    return all_embeddings

## Compute Embeddings

The code below computes the embeddings for all the images in our dataset. This step will take a few minutes for the MIT Indoor Scene Recognition dataset. There are over 10,000 images in the training set that we need to pass through DINOv2.

In [55]:
embeddings = compute_embeddings(files)

In [56]:
#get 1st dictionary key value
key = list(embeddings.keys())[0]
embeddings[key]

[[-4.69566535949707,
  2.0692639350891113,
  -2.584895372390747,
  -1.5161718130111694,
  0.02224058099091053,
  0.538525402545929,
  -0.11416690796613693,
  -2.0414535999298096,
  0.7016854286193848,
  3.519880533218384,
  0.6124308109283447,
  -3.493220567703247,
  -1.4799129962921143,
  -4.087428569793701,
  2.9313583374023438,
  -2.6530163288116455,
  -0.07802803069353104,
  -6.419325828552246,
  0.6758205890655518,
  -2.8354687690734863,
  4.1174702644348145,
  0.9184275269508362,
  2.8299458026885986,
  0.7693573236465454,
  0.2982948422431946,
  -0.26784273982048035,
  2.5362939834594727,
  3.678116798400879,
  0.9746991395950317,
  -5.1478271484375,
  0.22190290689468384,
  3.063937187194824,
  2.7596356868743896,
  1.6562373638153076,
  -3.6707515716552734,
  -1.9588758945465088,
  2.787867307662964,
  -0.517403781414032,
  -1.0838159322738647,
  3.082249879837036,
  -3.7270123958587646,
  -1.9567997455596924,
  -4.6668548583984375,
  2.4421305656433105,
  1.7438805103302002,


In [57]:
# Check the shape of embedding_list before reshaping
embedding_list = list(embeddings.values())
embedding_array = np.array(embedding_list)
print(f"Original shape: {embedding_array.shape}")
print(f"Total number of elements: {embedding_array.size}")

# Attempt to reshape
reshaped_array = embedding_array.reshape(-1, EMBEDDING_SIZE)
print(f"Reshaped array shape: {reshaped_array.shape}")

Original shape: (3544, 1, 384)
Total number of elements: 1360896
Reshaped array shape: (3544, 384)


## Train a Classification Model

The embeddings we have computed can be used as an input in a classification model. For this guide, we will be using SVM, a linear classification model.

Below, we make lists of both all of the embeddings we have computed and their associated labels. We then fit our model using those lists.

In [58]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

y = [labels[file] for file in files]
embedding_list = list(embeddings.values())
embedding_arr = np.array(embedding_list).reshape(-1, EMBEDDING_SIZE)
N_COMPONENTS = 30
N_ESTIMATORS = 120
MAX_DEPTH = 100
R_STATE = 21

#fit a svm model
def fit_svm_pca(embedding_list, y):
    clf = svm.SVC(gamma='scale')
    # Convert embedding_list to a 2D array
    embedding_array = np.array(embedding_list).reshape(len(embedding_list), -1)
    
    # Apply PCA to reduce dimensions to x principal components
    pca = PCA(n_components=N_COMPONENTS, random_state=R_STATE)
    embedding_list = pca.fit_transform(embedding_array)
    
    clf.fit(embedding_list, y)
    return clf, pca

clf_svm_pca, pca = fit_svm_pca(embedding_list, y)


#fit a svm model with pca
def fit_svm(embedding_list, y):
    clf = svm.SVC(gamma='scale')
    clf.fit(np.array(embedding_list).reshape(-1, EMBEDDING_SIZE), y)
    return clf

clf_svm = fit_svm(embedding_list, y)


#fit a random forest model
def fit_rf(embedding_list, y):
    rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=R_STATE) #70
    rf.fit(np.array(embedding_list).reshape(-1, EMBEDDING_SIZE), y)
    return rf

clf_rf = fit_rf(embedding_list, y)
#save this rf model


#fit a random forest model with pca
def fit_rf_pca(embedding_list, y):
    rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=R_STATE) #70

    # Convert embedding_list to a 2D array
    embedding_array = np.array(embedding_list).reshape(len(embedding_list), -1)
    
    # Apply PCA to reduce dimensions to x principal components
    pca = PCA(n_components=N_COMPONENTS, random_state=R_STATE)
    embedding_list = pca.fit_transform(embedding_array)

    rf.fit(embedding_list, y)
    return rf, pca

clf_rf_pca, rf_pca = fit_rf_pca(embedding_list, y)




In [59]:
import joblib
save_model = "no"
if save_model == "yes":

    joblib.dump(clf_rf_pca, 'models/rf_pca/best_rf_model_oct1st.pkl')
    joblib.dump(rf_pca, 'models/rf_pca/best_pca_model.pkl')

## Classify an Image

We now have a classifier we can use to classify images!

Change the `input_file` value below to the path of a file in the `valid` or `test` directories in the image dataset with which we have been working.

Then, run the cell to classify the image.

In [60]:
#any file in the folder title Now
TEST_FOLDER = "bear"
input_files = glob.glob(f'test/{TEST_FOLDER}/*.png')

for file in glob.glob(f'test/bull/*.png'):
    input_files.append(file)


In [61]:
input_files

['test/bear/chart_3574.png',
 'test/bear/chart_3661.png',
 'test/bear/chart_3695.png',
 'test/bear/chart_3610.png',
 'test/bear/chart_3649.png',
 'test/bear/chart_3630.png',
 'test/bear/chart_3682.png',
 'test/bear/chart_3596.png',
 'test/bear/chart_3604.png',
 'test/bear/chart_3697.png',
 'test/bear/chart_3607.png',
 'test/bear/chart_3690.png',
 'test/bear/chart_3645.png',
 'test/bear/chart_3640.png',
 'test/bear/chart_3617.png',
 'test/bear/chart_3683.png',
 'test/bear/chart_3600.png',
 'test/bear/chart_3616.png',
 'test/bear/chart_3585.png',
 'test/bear/chart_3657.png',
 'test/bear/chart_3672.png',
 'test/bear/chart_3691.png',
 'test/bear/chart_3648.png',
 'test/bear/chart_3614.png',
 'test/bear/chart_3659.png',
 'test/bear/chart_3637.png',
 'test/bear/chart_3652.png',
 'test/bear/chart_3595.png',
 'test/bear/chart_3653.png',
 'test/bear/chart_3606.png',
 'test/bear/chart_3622.png',
 'test/bear/chart_3655.png',
 'test/bear/chart_3599.png',
 'test/bear/chart_3611.png',
 'test/bear/ch

In [62]:
# Initialize an empty DataFrame
import pandas as pd

predictions_df = pd.DataFrame(columns=['image_file', 'svm_prediction', 'svm_pca_prediction', 'rf_prediction', 
                                       'rf_pca_prediction'])



for input_file in input_files:
    new_image = load_image(input_file)

    with torch.no_grad():
        new_embedding = dinov2_vits14(new_image.to(device))

        # Convert embedding to numpy array and reshape
        new_embedding_array = np.array(new_embedding[0].cpu()).reshape(1, -1)
        new_embedding_pca = pca.transform(new_embedding_array)
        
        # Generate with newly trained model the predictions
        svm_pca_prediction = clf_svm_pca.predict(new_embedding_pca)
        svm_prediction = clf_svm.predict(new_embedding_array)
        rf_prediction = clf_rf.predict(new_embedding_array)
        rf_pca_prediction = clf_rf_pca.predict(new_embedding_pca)



 

        # Add the predictions to the DataFrame using loc
        predictions_df.loc[len(predictions_df)] = [input_file, svm_prediction[0],svm_pca_prediction[0],
                                                    rf_prediction[0], rf_pca_prediction[0]]


/tmp/ipykernel_24480/305004702.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  new_embedding_array = np.array(new_embedding[0].cpu()).reshape(1, -1)
/tmp/ipykernel_24480/305004702.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  new_embedding_array = np.array(new_embedding[0].cpu()).reshape(1, -1)
/tmp/ipykernel_24480/305004702.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  new_embedding_array = np.array(new_embedding[0].cpu()).reshape(1, -1)
/tmp/ipykernel_24480/305004702.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must impl

In [63]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Extract true labels from the file paths
predictions_df['true_label'] = predictions_df['image_file'].apply(lambda x: 'bull' if 'bull' in x else 'bear')


# Compute accuracy for SVM and RF predictions
svm_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['svm_prediction'])
svm_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['svm_pca_prediction'])
rf_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['rf_prediction'])
rf_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['rf_pca_prediction'])


print('ALL')
print('--------------------------------------------------')
print(f"SVM Accuracy: {round(svm_accuracy, 2)}")
print(f"RF Accuracy: {round(rf_accuracy,2)}")
print(f"SVM PCA Accuracy: {round(svm_pca_accuracy,2)}")
print(f"RF PCA Accuracy: {round(rf_pca_accuracy,2)}")
print('--------------------------------------------------')


predictions_df

ALL
--------------------------------------------------
SVM Accuracy: 0.43
RF Accuracy: 0.54
SVM PCA Accuracy: 0.5
RF PCA Accuracy: 0.53
--------------------------------------------------


,image_file,svm_prediction,svm_pca_prediction,rf_prediction,rf_pca_prediction,true_label
0,test/bear/chart_3574.png,bull,bull,bull,bull,bear
1,test/bear/chart_3661.png,bear,bull,bull,bull,bear
2,test/bear/chart_3695.png,bull,bull,bear,bull,bear
3,test/bear/chart_3610.png,bull,bull,bull,bull,bear
4,test/bear/chart_3649.png,bull,bear,bear,bear,bear
...,...,...,...,...,...,...
143,test/bull/chart_2389.png,bear,bull,bull,bull,bull
144,test/bull/chart_2310.png,bear,bear,bear,bull,bull
145,test/bull/chart_2330.png,bear,bull,bear,bear,bull
146,test/bull/chart_2355.png,bull,bull,bull,bear,bull
